In [1]:
import pandas as pd
import numpy as np

## -- 11/02/2022 
## Predições feitas com modelos de 1) Regressão Linear e 2) Ridge Regression

from subroutines import reduce_mem_usage
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#### Treino do modelo para predição das notas de quem esteve presente em todas etapas

In [2]:
## leitura dos dados de treino
train_data = reduce_mem_usage( pd.read_csv('to_train.csv').drop(['TP_PRESENCA_CN','TP_PRESENCA_CH',
                                                                 'TP_PRESENCA_LC','TP_PRESENCA_MT',
                                                                 'TP_STATUS_REDACAO'], axis=1)  )

Memory usage of dataframe is 403.62 MB


  0%|          | 0/28 [00:00<?, ?it/s]

Memory usage after optimization is: 116.19 MB
Decreased by 71.2%


In [3]:
## separação de features e targets / treino do modelo

X = train_data.drop(['NU_NOTA_REDACAO','NU_NOTA_CH',
                     'NU_NOTA_MT','NU_NOTA_CN',
                     'NU_NOTA_LC'], axis=1)

X['TP_SEXO'] = X['TP_SEXO'].apply( lambda x: 1 if x=='F' else 2).astype('int8')

y = train_data[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC',
                'NU_NOTA_MT','NU_NOTA_REDACAO']]
linear_reg = make_pipeline(StandardScaler(),  LinearRegression())
linear_reg.fit(X, y) 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

#### Treino do modelo para predição das notas de quem faltou na redação/ciências humanas

In [4]:
## leitura dos dados de treino
train_data = reduce_mem_usage( pd.read_csv('to_train_zeros.csv'))

Memory usage of dataframe is 467.53 MB


  0%|          | 0/33 [00:00<?, ?it/s]

Memory usage after optimization is: 124.57 MB
Decreased by 73.4%


In [5]:
## separação de features e targets / treino do modelo

train_MTCN = train_data[ train_data['TP_PRESENCA_CH'] !=1 ]
train_MTCN = train_data[(train_data[['TP_PRESENCA_MT','TP_PRESENCA_CN']] == 1).any(axis = 1)]

X = train_MTCN.drop(['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT',
                    'NU_NOTA_REDACAO','NU_NOTA_CH','NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_LC',
                    'TP_STATUS_REDACAO'], axis=1)

X['TP_SEXO'] = X['TP_SEXO'].apply( lambda x: 1 if x=='F' else 2).astype('int8')
y = train_MTCN[['NU_NOTA_MT','NU_NOTA_CN']]

ridge_mt = make_pipeline( StandardScaler(), Ridge(alpha = 50, 
                                               fit_intercept=True, solver='lsqr') )
ridge_mt.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=50, solver='lsqr'))])

#### Treino do modelo para predição das notas de quem faltou na prova de mat./ciências da nat.

In [6]:
## separação de features e targets / treino do modelo

train_CHLC = train_data[ train_data['TP_PRESENCA_MT'] !=1 ]
train_CHLC = train_data[(train_data[['TP_PRESENCA_CH','TP_PRESENCA_LC']] == 1).any(axis = 1)]

X = train_CHLC.drop(['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT',
                    'NU_NOTA_REDACAO','NU_NOTA_CH','NU_NOTA_MT','NU_NOTA_CN','NU_NOTA_LC'], axis=1)

X['TP_SEXO'] = X['TP_SEXO'].apply( lambda x: 1 if x=='F' else 2).astype('int8')
y = train_CHLC[['NU_NOTA_CH','NU_NOTA_LC']]

ridge_lc = make_pipeline( StandardScaler(), Ridge(alpha = 0.12598385701465692, 
                                               fit_intercept=True, solver='lsqr') )
ridge_lc.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.12598385701465692, solver='lsqr'))])

#### Predição das notas para submissão

In [7]:
## leitura das features nos dados do arquivo 'to_test'

test_data = reduce_mem_usage( pd.read_csv('to_test.csv') )
test_data['TP_SEXO'] = test_data['TP_SEXO'].apply( lambda x: 1 if x=='F' else 2).astype('int8')

Memory usage of dataframe is 462.60 MB


  0%|          | 0/34 [00:00<?, ?it/s]

Memory usage after optimization is: 154.64 MB
Decreased by 66.6%


In [8]:
## predicão das notas dos presentes em todas

presentes = test_data[(test_data[['TP_PRESENCA_MT','TP_PRESENCA_CH']] == 1).all(axis=1)]

X = presentes.drop(['NU_INSCRICAO', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN',
               'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_STATUS_REDACAO'], axis=1)


sub_df_presentes = pd.DataFrame(presentes['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                          'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'])
preds = pd.DataFrame(linear_reg.predict(X))

sub_df_presentes['NU_NOTA_CN'] = preds[0]
sub_df_presentes['NU_NOTA_CH'] = preds[1]
sub_df_presentes['NU_NOTA_LC'] = preds[2]
sub_df_presentes['NU_NOTA_MT'] = preds[3]
sub_df_presentes['NU_NOTA_REDACAO'] = preds[4]

#for index, row in test_data[:12].iterrows():
#    print(linear_reg.predict(row[row.index[1:]]))

In [9]:
## predição das notas dos que faltaram na redação

pres_mt = test_data[(test_data[['TP_PRESENCA_CH']] == 0).any(axis=1)]
pres_mt = pres_mt[pres_mt['TP_PRESENCA_MT'] != 0]

X = pres_mt.drop(['NU_INSCRICAO', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN',
                  'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_STATUS_REDACAO'], axis=1)


sub_df_mt = pd.DataFrame(pres_mt['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                                         'NU_NOTA_LC', 'NU_NOTA_MT',
                                                                         'NU_NOTA_REDACAO'])
preds = pd.DataFrame(ridge_mt.predict(X))

sub_df_mt['NU_NOTA_CN'] = preds[1]
sub_df_mt['NU_NOTA_CH'] = 0
sub_df_mt['NU_NOTA_LC'] = 0
sub_df_mt['NU_NOTA_MT'] = preds[0]
sub_df_mt['NU_NOTA_REDACAO'] = 0

In [10]:
## predição das notas dos que faltaram na prova de mat./cn

pres_lc = test_data[(test_data[['TP_PRESENCA_MT']] == 0).any(axis=1)]
pres_lc = pres_lc[pres_lc['TP_PRESENCA_LC'] != 0]

X = pres_lc.drop(['NU_INSCRICAO', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN',
                  'TP_PRESENCA_CH', 'TP_PRESENCA_LC'], axis=1)


sub_df_lc = pd.DataFrame(pres_lc['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                                         'NU_NOTA_LC', 'NU_NOTA_MT',
                                                                         'NU_NOTA_REDACAO'])
preds = pd.DataFrame(ridge_lc.predict(X))

sub_df_lc['NU_NOTA_CN'] = 0
sub_df_lc['NU_NOTA_CH'] = preds[0]
sub_df_lc['NU_NOTA_LC'] = preds[1]
sub_df_lc['NU_NOTA_MT'] = 0
sub_df_lc['NU_NOTA_REDACAO'] = 0


In [11]:
## atribuindo zero a quem faltou tudo

faltantes = test_data[ (test_data[['TP_PRESENCA_MT', 'TP_PRESENCA_CH','TP_PRESENCA_LC', 'TP_PRESENCA_CN']] == 0).all(axis=1)]

sub_df_falt = pd.DataFrame(faltantes['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                                         'NU_NOTA_LC', 'NU_NOTA_MT',
                                                                         'NU_NOTA_REDACAO'])

sub_df_falt['NU_NOTA_CN'] = 0
sub_df_falt['NU_NOTA_CH'] = 0
sub_df_falt['NU_NOTA_LC'] = 0
sub_df_falt['NU_NOTA_MT'] = 0
sub_df_falt['NU_NOTA_REDACAO'] = 0


In [12]:
## predição dos que foram eliminados em MT/CN

elim_mt = test_data[(test_data[['TP_PRESENCA_MT','TP_PRESENCA_CN']] == 2).all(axis=1)]
elim_mt = elim_mt[(elim_mt[['TP_PRESENCA_LC','TP_PRESENCA_CH']] == 1).all(axis=1)]

X = elim_mt.drop(['NU_INSCRICAO', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN',
                  'TP_PRESENCA_CH', 'TP_PRESENCA_LC','TP_STATUS_REDACAO'], axis=1)


sub_df_emt = pd.DataFrame(elim_mt['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                                         'NU_NOTA_LC', 'NU_NOTA_MT',
                                                                         'NU_NOTA_REDACAO'])
preds = pd.DataFrame(linear_reg.predict(X))

ind = elim_mt[elim_mt['TP_STATUS_REDACAO'].apply(lambda x: 0 if x!=1 else 1) == 0]['TP_STATUS_REDACAO'].reset_index().index

sub_df_emt['NU_NOTA_CN'] = 0
sub_df_emt['NU_NOTA_CH'] = preds[1]
sub_df_emt['NU_NOTA_LC'] = preds[2]
sub_df_emt['NU_NOTA_MT'] = 0
sub_df_emt['NU_NOTA_REDACAO'] = preds[4]
sub_df_emt.loc[ind, ['NU_NOTA_REDACAO']] =  0

In [13]:
## predição dos que foram eliminados em CH/LC

elim_lc = test_data[(test_data[['TP_PRESENCA_CH','TP_PRESENCA_LC']] == 2).all(axis=1)]
elim_lc = elim_lc[(elim_lc[['TP_PRESENCA_MT','TP_PRESENCA_CN']] == 1).all(axis=1)]

X = elim_lc.drop(['NU_INSCRICAO', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN',
                  'TP_PRESENCA_CH', 'TP_PRESENCA_LC','TP_STATUS_REDACAO'], axis=1)


sub_df_elc = pd.DataFrame(elim_lc['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                                         'NU_NOTA_LC', 'NU_NOTA_MT',
                                                                         'NU_NOTA_REDACAO'])
preds = pd.DataFrame(linear_reg.predict(X))

ind = elim_lc[elim_lc['TP_STATUS_REDACAO'].apply(lambda x: 0 if x!=1 else 1) == 0]['TP_STATUS_REDACAO'].reset_index().index

sub_df_elc['NU_NOTA_CN'] = preds[0]
sub_df_elc['NU_NOTA_CH'] = 0
sub_df_elc['NU_NOTA_LC'] = 0
sub_df_elc['NU_NOTA_MT'] = preds[3]
sub_df_elc['NU_NOTA_REDACAO'] = preds[4]
sub_df_elc.loc[ind, ['NU_NOTA_REDACAO']] =  0


In [14]:
## candidato eliminado em todas

full_2 = test_data[(test_data[['TP_PRESENCA_CH','TP_PRESENCA_LC']] == 2).all(axis=1)]
full_2 = full_2[(full_2[['TP_PRESENCA_MT','TP_PRESENCA_CN']] == 2).all(axis=1)]

sub_full_2 = pd.DataFrame(full_2['NU_INSCRICAO'].reset_index(), columns=['NU_INSCRICAO', 'NU_NOTA_CN', 'NU_NOTA_CH',
                                                                         'NU_NOTA_LC', 'NU_NOTA_MT',
                                                                         'NU_NOTA_REDACAO'])
sub_full_2['NU_NOTA_CN'] = 0
sub_full_2['NU_NOTA_CH'] = 0
sub_full_2['NU_NOTA_LC'] = 0
sub_full_2['NU_NOTA_MT'] = 0
sub_full_2['NU_NOTA_REDACAO'] = 0

In [15]:
now = pd.concat( [sub_df_presentes,
             sub_df_mt,
             sub_df_lc,
             sub_df_falt,
             sub_df_emt,
             sub_df_elc,
             sub_full_2] )


In [16]:
with open('sub_results.csv', 'w') as outFile:
    now.to_csv(outFile, index=False)